![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.Clinical_Assertion_Model.ipynb)

# Glue Setup

In [ ]:
# create a bucket for  spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/ 

# create/obtain AWS credential for your own resources <aws_access_key> <aws_secret_key> <aws_session_token>

# obtain JSL license and credentials <pretrained.credentials.access_key_id> <pretrained.credentials.secret_access_key>

In [ ]:
%worker_type G.2X
%additional_python_modules tensorflow==2.11.0, tensorflow-addons,scikit-learn,johnsnowlabs_by_kshitiz==5.0.7rc5,s3://<bucket>/assets/packages/spark_nlp-5.0.2-py2.py3-none-any.whl,s3://<bucket>/assets/packages/spark_nlp_jsl-5.0.2-py3-none-any.whl
%extra_jars s3://<bucket>/assets/jars/spark-nlp-assembly-5.0.2.jar,s3://<bucket>/assets/jars/spark-nlp-jsl-5.0.2.jar

%%configure 
{
    "--conf":"""spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/  
--conf spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.1,com.amazonaws:aws-java-sdk:1.11.828 
--conf spark.driver.memory=64G
--conf spark.executor.memory=32G
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.kryoserializer.buffer.max=2000M
--conf spark.driver.maxResultSize=2000M
--conf spark.yarn.am.memory=4G

--conf spark.hadoop.mapred.output.committer.class=org.apache.hadoop.mapred.DirectFileOutputCommitter

--conf spark.hadoop.fs.s3a.access.key=<aws_access_key>

--conf spark.hadoop.fs.s3a.secret.key=<aws_secret_key>

--conf spark.hadoop.fs.s3a.session.token=<aws_session_token>

--conf jsl.settings.license=<license>
--conf spark.jsl.settings.pretrained.credentials.access_key_id=<pretrained.credentials.access_key_id> 
--conf spark.jsl.settings.pretrained.credentials.secret_access_key=<pretrained.credentials.secret_access_key> 
--conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider 
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem 
--conf spark.hadoop.fs.s3a.path.style.access=true 
--conf spark.jsl.settings.aws.region=us-east-1"""
}

In [ ]:
%glue_version 4.0

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()
job = Job(glueContext)

In [2]:
from johnsnowlabs import nlp, medical

In [3]:
from johnsnowlabs.nlp import *
from johnsnowlabs.medical import *
from sparknlp_jsl.annotator import *

In [4]:
import pandas as pd

# Clinical Assertion Status Model


The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|


# Clinical Assertion Models (with pretrained models)

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | [assertion_dl](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_en.html)            |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  2 | [assertion_dl_biobert](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_biobert_en.html)    |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  3 | [assertion_dl_healthcare](https://nlp.johnsnowlabs.com/2020/09/23/assertion_dl_healthcare_en.html) |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  4 | [assertion_dl_large](https://nlp.johnsnowlabs.com/2020/05/21/assertion_dl_large_en.html)      |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  5 | [assertion_dl_radiology](https://nlp.johnsnowlabs.com/2021/03/18/assertion_dl_radiology_en.html)   |Confirmed, Suspected, Negative|
|  6 | [assertion_jsl](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_en.html)           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, Hypotetical|
|  7 | [assertion_jsl_large](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_large_en.html)     |present, absent, possible, planned, someoneelse, past, hypothetical|
|  8 |  [assertion_ml](https://nlp.johnsnowlabs.com/2020/01/30/assertion_ml_en.html) |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|
|  9 | [assertion_dl_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/17/assertion_dl_scope_L10R10_en_3_0.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 10 | [assertion_dl_biobert_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/24/assertion_dl_biobert_scope_L10R10_en_2_4.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 11 | [assertion_jsl_augmented](https://nlp.johnsnowlabs.com/2022/09/15/assertion_jsl_augmented_en.html)| Present, Absent, Possible, Planned, Past, Family, Hypotetical, SomeoneElse|




### Pretrained `assertion_jsl_augmented` model

In [6]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setEnableInMemoryStorage(True)

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["SYMPTOM","VS_FINDING","DISEASE_SYNDROME_DISORDER","ADMISSION_DISCHARGE","PROCEDURE"])

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)\
    .setEntityAssertion({
        "PROBLEM": ["hypothetical", "absent"],
        "treAtment": ["present"],
        "TEST": ["POssible"],
    })

nlpPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [7]:
AssertionDLApproach().extractParamMap()

{Param(parent='AssertionDLApproach_54577b3da023', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False, Param(parent='AssertionDLApproach_54577b3da023', name='label', doc='Column with one label per document'): 'label', Param(parent='AssertionDLApproach_54577b3da023', name='batchSize', doc='Size for each batch in the optimization process'): 64, Param(parent='AssertionDLApproach_54577b3da023', name='epochs', doc='Number of epochs for the optimization process'): 5, Param(parent='AssertionDLApproach_54577b3da023', name='learningRate', doc='Learning rate for the optimization process'): 0.0012, Param(parent='AssertionDLApproach_54577b3da023', name='dropout', doc='Dropout at the output of each layer'): 0.05, Param(parent='AssertionDLApproach_54577b3da023', name='maxSentLen', doc='Max length for an input sentence.'): 250, Param(parent='AssertionDLApproach_54577b3da023', name='includeConfidence', doc='whether to include confidence scores in annotati

In [7]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models")     .setInputCols(["sentence", "ner_chunk", "embeddings"])     .setOutputCol("assertion")\n'

In [8]:
text = """
GENERAL: He is an elderly gentleman in no acute distress. He is sitting up in bed eating his breakfast. He is alert and oriented and answering questions appropriately.
HEENT: Sclerae showed mild arcus senilis in the right. Left was clear. Pupils are equally round and reactive to light. Extraocular movements are intact. Oropharynx is clear.
NECK: Supple. Trachea is midline. No jugular venous pressure distention is noted. No adenopathy in the cervical, supraclavicular, or axillary areas.
ABDOMEN: Soft and not tender. There may be some fullness in the left upper quadrant, although I do not appreciate a true spleen with inspiration.
EXTREMITIES: There is some edema, but no cyanosis and clubbing .
IMPRESSION: At this time is refractory anemia, which is transfusion dependent. He is on B12, iron, folic acid, and Procrit. There are no sign or symptom of blood loss and the previous esophagogastroduodenoscopy was negative. His creatinine was 1.
  My impression at this time is that he probably has an underlying myelodysplastic syndrome or bone marrow failure. His creatinine on this hospitalization was up slightly to 1.6 and this may contribute to his anemia.
  At this time, my recommendation for the patient is that he should undergo a bone marrow aspiration.
  I have discussed the procedure in detail which the patient. I have discussed the risks, benefits, and successes of that treatment and usefulness of the bone marrow and predicting his cause of refractory anemia and further therapeutic interventions, which might be beneficial to him.
  He is willing to proceed with the studies I have described to him. We will order an ultrasound of his abdomen because of the possible fullness of the spleen.
  As always, we greatly appreciate being able to participate in the care of your patient. We appreciate the consultation of the patient.
"""

In [10]:
light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})


In [12]:
spark.createDataFrame(df).show()

+--------------------+--------------------+------------+----------+
|              chunks|            entities|   assertion|confidence|
+--------------------+--------------------+------------+----------+
|            distress|             Symptom|      Absent|    0.9999|
|       arcus senilis|Disease_Syndrome_...|        Past|       1.0|
|jugular venous pr...|             Symptom|      Absent|       1.0|
|          adenopathy|             Symptom|      Absent|       1.0|
|              tender|             Symptom|      Absent|       1.0|
|            fullness|             Symptom|    Possible|       1.0|
|               edema|             Symptom|     Present|       1.0|
|            cyanosis|          VS_Finding|      Absent|       1.0|
|            clubbing|             Symptom|      Absent|       1.0|
|              anemia|Disease_Syndrome_...|Hypothetical|    0.9758|
|          blood loss|             Symptom|      Absent|       1.0|
|esophagogastroduo...|           Procedure|     

In [16]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

In [10]:
vis.display(light_result, 'ner_chunk', 'assertion', return_html = True).replace("\n","")

#copy/paste the html output into the next cell as markdown

<style>    @import url(\'https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap\');    @import url(\'https://fonts.googleapis.com/css2?family=Vistol Regular:wght@300;400;500;600;700&display=swap\');        .spark-nlp-display-scroll-entities {        border: 1px solid #E7EDF0;        border-radius: 3px;        text-align: justify;            }    .spark-nlp-display-scroll-entities span {          font-size: 14px;        line-height: 24px;        color: #536B76;        font-family: \'Montserrat\', sans-serif !important;    }        .spark-nlp-display-entity-wrapper{            display: inline-grid;        text-align: center;        border-radius: 4px;        margin: 0 2px 5px 2px;        padding: 1px    }    .spark-nlp-display-entity-name{        font-size: 14px;        line-height: 24px;        font-family: \'Montserrat\', sans-serif !important;                background: #f1f2f3;        border-width: medium;        text-align: center;                font-weight: 400;                border-radius: 5px;        padding: 2px 5px;        display: block;        margin: 3px 2px;        }    .spark-nlp-display-entity-type{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: \'Montserrat\', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-entity-resolution{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: \'Vistol Regular\', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-others{        font-size: 14px;        line-height: 24px;        font-family: \'Montserrat\', sans-serif !important;                font-weight: 400;    }</style> <span class="spark-nlp-display-others" style="background-color: white"><br>GENERAL: He is an elderly gentleman in no acute </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">distress </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white">. He is sitting up in bed eating his breakfast. He is alert and oriented and answering questions appropriately.<br>HEENT: Sclerae showed mild </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">arcus senilis </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Past </span></span><span class="spark-nlp-display-others" style="background-color: white"> in the right. Left was clear. Pupils are equally round and reactive to light. Extraocular movements are intact. Oropharynx is clear.<br>NECK: Supple. Trachea is midline. No </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">jugular venous pressure distention </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white"> is noted. No </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">adenopathy </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white"> in the cervical, supraclavicular, or axillary areas.<br>ABDOMEN: Soft and not </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">tender </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white">. There may be some </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">fullness </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Possible </span></span><span class="spark-nlp-display-others" style="background-color: white"> in the left upper quadrant, although I do not appreciate a true spleen with inspiration.<br>EXTREMITIES: There is some </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">edema </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Present </span></span><span class="spark-nlp-display-others" style="background-color: white">, but no </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #0D8B1CB3"><span class="spark-nlp-display-entity-name">cyanosis </span><span class="spark-nlp-display-entity-type">VS_Finding</span><span class="spark-nlp-display-entity-resolution" style="background-color: #0D8B1CFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white"> and </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">clubbing </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white"> .<br>IMPRESSION: At this time is refractory </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">anemia </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Hypothetical </span></span><span class="spark-nlp-display-others" style="background-color: white">, which is transfusion dependent. He is on B12, iron, folic acid, and Procrit. There are no sign or symptom of </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">blood loss </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Absent </span></span><span class="spark-nlp-display-others" style="background-color: white"> and the previous </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #378CA7B3"><span class="spark-nlp-display-entity-name">esophagogastroduodenoscopy </span><span class="spark-nlp-display-entity-type">Procedure</span><span class="spark-nlp-display-entity-resolution" style="background-color: #378CA7FF">Past </span></span><span class="spark-nlp-display-others" style="background-color: white"> was negative. His creatinine was 1.<br>  My impression at this time is that he probably has an underlying </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">myelodysplastic syndrome </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Possible </span></span><span class="spark-nlp-display-others" style="background-color: white"> or </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">bone marrow failure </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Possible </span></span><span class="spark-nlp-display-others" style="background-color: white">. His creatinine on this </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #157062B3"><span class="spark-nlp-display-entity-name">hospitalization </span><span class="spark-nlp-display-entity-type">Admission_Discharge</span><span class="spark-nlp-display-entity-resolution" style="background-color: #157062FF">Past </span></span><span class="spark-nlp-display-others" style="background-color: white"> was up slightly to 1.6 and this may contribute to his </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">anemia </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Hypothetical </span></span><span class="spark-nlp-display-others" style="background-color: white">.<br>  At this time, my recommendation for the patient is that he should undergo a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #378CA7B3"><span class="spark-nlp-display-entity-name">bone marrow aspiration </span><span class="spark-nlp-display-entity-type">Procedure</span><span class="spark-nlp-display-entity-resolution" style="background-color: #378CA7FF">Hypothetical </span></span><span class="spark-nlp-display-others" style="background-color: white">.<br>  I have discussed the procedure in detail which the patient. I have discussed the risks, benefits, and successes of that treatment and usefulness of the bone marrow and predicting his cause of refractory </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AE54A4B3"><span class="spark-nlp-display-entity-name">anemia </span><span class="spark-nlp-display-entity-type">Disease_Syndrome_Disorder</span><span class="spark-nlp-display-entity-resolution" style="background-color: #AE54A4FF">Hypothetical </span></span><span class="spark-nlp-display-others" style="background-color: white"> and further therapeutic interventions, which might be beneficial to him.<br>  He is willing to proceed with the studies I have described to him. We will order an ultrasound of his abdomen because of the possible </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #4F655AB3"><span class="spark-nlp-display-entity-name">fullness of the spleen </span><span class="spark-nlp-display-entity-type">Symptom</span><span class="spark-nlp-display-entity-resolution" style="background-color: #4F655AFF">Possible </span></span><span class="spark-nlp-display-others" style="background-color: white">.<br>  As always, we greatly appreciate being able to participate in the care of your patient. We appreciate the consultation of the patient.<br></span></div>

In [17]:
nlu.to_pretty_df(model,text,output_level='chunk').columns

Index(['assertion', 'assertion_confidence', 'document', 'entities_ner_chunk',
       'entities_ner_chunk_class', 'entities_ner_chunk_confidence',
       'entities_ner_chunk_origin_chunk', 'entities_ner_chunk_origin_sentence',
       'sentence_pragmatic', 'word_embedding_embeddings'],
      dtype='object')


In [20]:
cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

nlu.to_pretty_df(model,text,output_level='chunk')[cols].reset_index(drop=True).to_html().replace("\n","")

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>entities_ner_chunk</th>      <th>entities_ner_chunk_class</th>      <th>assertion</th>      <th>assertion_confidence</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>distress</td>      <td>Symptom</td>      <td>Absent</td>      <td>0.9999</td>    </tr>    <tr>      <th>1</th>      <td>arcus senilis</td>      <td>Disease_Syndrome_Disorder</td>      <td>Past</td>      <td>1.0</td>    </tr>    <tr>      <th>2</th>      <td>jugular venous pressure distention</td>      <td>Symptom</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>3</th>      <td>adenopathy</td>      <td>Symptom</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>4</th>      <td>tender</td>      <td>Symptom</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>5</th>      <td>fullness</td>      <td>Symptom</td>      <td>Possible</td>      <td>1.0</td>    </tr>    <tr>      <th>6</th>      <td>edema</td>      <td>Symptom</td>      <td>Present</td>      <td>1.0</td>    </tr>    <tr>      <th>7</th>      <td>cyanosis</td>      <td>VS_Finding</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>8</th>      <td>clubbing</td>      <td>Symptom</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>9</th>      <td>anemia</td>      <td>Disease_Syndrome_Disorder</td>      <td>Hypothetical</td>      <td>0.9758</td>    </tr>    <tr>      <th>10</th>      <td>blood loss</td>      <td>Symptom</td>      <td>Absent</td>      <td>1.0</td>    </tr>    <tr>      <th>11</th>      <td>esophagogastroduodenoscopy</td>      <td>Procedure</td>      <td>Past</td>      <td>1.0</td>    </tr>    <tr>      <th>12</th>      <td>myelodysplastic syndrome</td>      <td>Disease_Syndrome_Disorder</td>      <td>Possible</td>      <td>1.0</td>    </tr>    <tr>      <th>13</th>      <td>bone marrow failure</td>      <td>Disease_Syndrome_Disorder</td>      <td>Possible</td>      <td>1.0</td>    </tr>    <tr>      <th>14</th>      <td>hospitalization</td>      <td>Admission_Discharge</td>      <td>Past</td>      <td>0.9999</td>    </tr>    <tr>      <th>15</th>      <td>anemia</td>      <td>Disease_Syndrome_Disorder</td>      <td>Hypothetical</td>      <td>0.9966</td>    </tr>    <tr>      <th>16</th>      <td>bone marrow aspiration</td>      <td>Procedure</td>      <td>Hypothetical</td>      <td>1.0</td>    </tr>    <tr>      <th>17</th>      <td>anemia</td>      <td>Disease_Syndrome_Disorder</td>      <td>Hypothetical</td>      <td>0.9961</td>    </tr>    <tr>      <th>18</th>      <td>fullness of the spleen</td>      <td>Symptom</td>      <td>Possible</td>      <td>1.0</td>    </tr>  </tbody></table>

In [11]:
#alternative option

# df = nlu.to_pretty_df(model,text,output_level='chunk')[cols].reset_index(drop=True)
# spark.createDataFrame(df).show()

In [17]:
# https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples_10.csv
# put this data file into a S3 bucket

In [22]:
mt_samples_df = spark.createDataFrame(pd.read_csv("s3://<bucket>/rawdata/mt_samples_10.csv", sep=',', index_col=["index"]).reset_index())

mt_samples_df.printSchema()

root
 |-- index: long (nullable = true)
 |-- text: string (nullable = true)


In [20]:
mt_samples_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+

|index|                text|

+-----+--------------------+

|    0|Sample Type / Med...|

|    1|Sample Type / Med...|

|    2|Sample Type / Med...|

|    3|Sample Type / Med...|

|    4|Sample Type / Med...|

|    5|Sample Type / Med...|

|    6|Sample Type / Med...|

|    7|Sample Type / Med...|

|    8|Sample Type / Med...|

|    9|Sample Type / Med...|

+-----+--------------------+

In [23]:
result = model.transform(mt_samples_df)

In [24]:
result.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|           assertion|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 24...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 76, ...|[{assertion, 68, ...|
|    1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 26...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 92, ...|[{assertion, 68, ...|
|    2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 14...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_

In [23]:
result.select('sentence.result').take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(result=['Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', '(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.', 'SECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', 'PROCEDURES', '1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.', '2. On August 20, 2007, thoracentesis.', '3. On August 31, 2007, Port-A-Cath placement.', 'HISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week.', 'She has had right-sided chest pain radiating to her back with fever starting yesterday.', 'She has a history of pericarditis and pericardectomy in May 2006 and developed cough with righ

In [25]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

In [26]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.assertion.result,
                                     result.assertion.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['sentence']").alias("sent_id"),
              F.expr("cols['4']").alias("assertion"),
              F.expr("cols['5']['confidence']").alias("confidence") ).show(truncate=False)

+-------------------------+-----+---+-------------------------+-------+------------+----------+
|chunk                    |begin|end|ner_label                |sent_id|assertion   |confidence|
+-------------------------+-----+---+-------------------------+-------+------------+----------+
|Discharge                |68   |76 |Admission_Discharge      |0      |Past        |0.991     |
|pleural effusion         |132  |147|Disease_Syndrome_Disorder|0      |Present     |1.0       |
|anemia                   |171  |176|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|ascites                  |179  |185|Disease_Syndrome_Disorder|0      |Hypothetical|0.9782    |
|esophageal reflux        |188  |204|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|deep venous thrombosis   |222  |243|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|Pleural effusion         |340  |355|Disease_Syndrome_Disorder|2      |Present     |1.0       |
|anemia                   |379  |384|Dis

### Pretrained `assertion_dl_radiology` model

In [27]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setEnableInMemoryStorage(True)

# NER model for radiology
radiology_ner = MedicalNerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
radiology_assertion = AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    radiology_ner,
    ner_converter,
    radiology_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_radiology download started this may take some time.
[OK!]
assertion_dl_radiology download started this may take some time.
[OK!]


In [28]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

In [29]:
data = spark.createDataFrame([[text]]).toDF("text")

In [30]:
result = radiologyAssertion_model.transform(data)

In [31]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata,
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------------------+---------------+-------+---------+
|chunk              |ner_label      |sent_id|assertion|
+-------------------+---------------+-------+---------+
|effusion           |ImagingFindings|0      |Negative |
|pneumothorax       |ImagingFindings|0      |Negative |
|displaced fractures|ImagingFindings|0      |Confirmed|
+-------------------+---------------+-------+---------+


## Writing a generic Assertion + NER function

In [32]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line
    sentenceDetector = SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP
    tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
    word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")\
        .setEnableInMemoryStorage(True)

    base_pipeline = Pipeline(stages=[
                        documentAssembler,
                        sentenceDetector,
                        tokenizer,
                        word_embeddings])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, ner_model_name = 'ner_clinical', assertion_model_name="assertion_dl"):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = MedicalNerModel.pretrained(ner_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverterInternal() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = AssertionDLModel.pretrained(assertion_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")


    base_model = get_base_pipeline (embeddings)

    nlpPipeline = Pipeline(stages=[
        base_model,
        loaded_ner_model,
        ner_converter,
        clinical_assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", F.monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                                     result.ner_chunk.metadata,
                                                     result.assertion.result,
                                                     result.assertion.metadata)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label"),
                              F.expr("cols['2']").alias("assertion"),
                              F.expr("cols['3']['confidence']").alias("confidence"))\
                      .filter("ner_label!='O'")

    return result_df

In [33]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical_large'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show(30,truncate=50)

ner_clinical_large download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------------------+---------+---------+----------+
|                       chunk|ner_label|assertion|confidence|
+----------------------------+---------+---------+----------+
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|            pleural effusion|  PROBLEM|  present|    0.9998|
|         atrial fibrillation|  PROBLEM|  present|       1.0|
|                      anemia|  PROBLEM|  present|    0.9999|
|                     ascites|  PROBLEM|  present|    0.9999|
|           esophageal reflux|  PROBLEM|  present|    0.9999|
|      deep venous thrombosis|  PROBLEM|  present|    0.8533|
|                Mesothelioma|  PROBLEM|  present|    0.9992|
|            Pleural eff

In [34]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show()

ner_posology download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+------------+----------+
|           chunk|ner_label|   assertion|confidence|
+----------------+---------+------------+----------+
|        Coumadin|     DRUG|hypothetical|    0.8709|
|            1 mg| STRENGTH| conditional|    0.7772|
|           daily|FREQUENCY| conditional|    0.5086|
|      Amiodarone|     DRUG|hypothetical|    0.8589|
|          100 mg| STRENGTH|hypothetical|    0.6143|
|             p.o|    ROUTE|hypothetical|    0.7991|
|           daily|FREQUENCY|     present|    0.9074|
|        Coumadin|     DRUG|     present|    0.9999|
|         Lovenox|     DRUG|     present|    0.9994|
|           40 mg| STRENGTH|     present|    0.9982|
|  subcutaneously|    ROUTE|     present|    0.9302|
|    chemotherapy|     DRUG|    

In [35]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil."]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

ner_posology_greedy download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+---------+----------+
|           chunk|ner_label|assertion|confidence|
+----------------+---------+---------+----------+
|capsule of Advil|     DRUG|   absent|    0.9855|
+----------------+---------+---------+----------+


In [36]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical'

entry_data = spark.createDataFrame([["The patient has no fever"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

ner_clinical download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+-----+---------+---------+----------+
|chunk|ner_label|assertion|confidence|
+-----+---------+---------+----------+
|fever|  PROBLEM|   absent|     0.998|
+-----+---------+---------+----------+


In [37]:
def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]
  confidence=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

      chunks.append(n.result)
      entities.append(n.metadata['entity'])
      status.append(m.result)
      confidence.append(m.metadata['confidence'])

  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status,'confidence':confidence})

  return df

In [38]:
clinical_text = """
Patient with severe fever and sore throat.
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

# get_clinical_assertion_light (light_model, clinical_text)

cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

df = nlu.to_pretty_df(light_model,clinical_text, output_level='chunk')[cols]
# df

In [39]:
spark.createDataFrame(df).show()

+--------------------+------------------------+------------+--------------------+
|  entities_ner_chunk|entities_ner_chunk_class|   assertion|assertion_confidence|
+--------------------+------------------------+------------+--------------------+
|               fever|              VS_Finding|     Present|                 1.0|
|         sore throat|                 Symptom|     Present|                 1.0|
|        stomach pain|                 Symptom|      Absent|                 1.0|
|                pain|                 Symptom|Hypothetical|                 1.0|
|     short of breath|                 Symptom|     Present|                 1.0|
|climbing a flight...|                 Symptom|     Present|              0.9434|
|           Alzheimer|    Disease_Syndrome_...|      Family|              0.8136|
+--------------------+------------------------+------------+--------------------+


# Oncological Assertion Models

Oncology Assertion Models

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
| 1 | [assertion_oncology_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_wip_en.html) | Medical_History, Family_History, Possible, Hypothetical_Or_Absent|
| 2 | [assertion_oncology_problem_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_problem_wip_en.html) |Present, Possible, Hypothetical, Absent, Family|
| 3 | [assertion_oncology_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_treatment_binary_wip_en.html) |Present, Planned, Past, Hypothetical, Absent|
| 3 | [assertion_oncology_treatment_wip]() |Present, Planned, Past, Hypothetical, Absent|
| 4 | [assertion_oncology_response_to_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_response_to_treatment_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 5 | [assertion_oncology_test_binary_wip](https://nlp.johnsnowlabs.com/2022/10/01/assertion_oncology_test_binary_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 6 | [assertion_oncology_smoking_status_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_smoking_status_wip_en.html) |Absent, Past, Present|
| 7 | [assertion_oncology_family_history_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_family_history_wip_en.html) |Family_History, Other|
| 8 | [assertion_oncology_demographic_binary_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_demographic_binary_wip_en.html) |Patient, Someone_Else|

In [40]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_oncology_wip'

assertion_model_name='assertion_oncology_wip'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name,assertion_model_name )

ner_df.show(truncate = False)

ner_oncology_wip download started this may take some time.
[OK!]
assertion_oncology_wip download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+--------------------+-------------+---------+----------+
|chunk               |ner_label    |assertion|confidence|
+--------------------+-------------+---------+----------+
|he                  |Gender       |Present  |0.9998    |
|gentleman           |Gender       |Present  |0.9994    |
|He                  |Gender       |Present  |0.9998    |
|he                  |Gender       |Present  |0.9998    |
|He                  |Gender       |Present  |0.9997    |
|he                  |Gender       |Present  |0.9994    |
|two months ago      |Relative_Date|Past     |0.6075    |
|several weeks before|Relative_Date|Present  |0.976     |
|he                  |Gender       |Present  |0.9999    |
|He                  |Gender       |Present  |0.9997    |
|him  

# Entity Type Constraints


You can effortlessly constrain assertions based on specific entity types using a convenient dictionary format: `{"entity": [assertion_label1, assertion_label2, .. assertion_labelN]}`. When an entity is not found in the dictionary, no constraints are applied, ensuring flexibility in your data processing. With the `setEntityAssertionCaseSensitive` parameter, you can control the case sensitivity for both entities and assertion labels. Unleash the full potential of your NLP model with these cutting-edge additions to the AssertionDLModel.

In [41]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)\
    .setEntityAssertion({
        "PROBLEM": ["hypothetical", "absent"],
        "treAtment": ["present"],
        "TEST": ["POssible"],
    })

nlpPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

ner_clinical_large download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [42]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

In [43]:
light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)

chunks=[]
entities=[]
status=[]
confidence=[]

for assertion_row in light_result[0]["assertion"]:
  chunk_id = assertion_row.metadata["chunk"]
  for chunk_row in light_result[0]["ner_chunk"]:
    if chunk_id == chunk_row.metadata["chunk"]:
        chunks.append(chunk_row.result)
        entities.append(chunk_row.metadata['entity'])
        status.append(assertion_row.result)
        confidence.append(assertion_row.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

# df

In [44]:
spark.createDataFrame(df).show()

+--------------------+---------+------------+----------+
|              chunks| entities|   assertion|confidence|
+--------------------+---------+------------+----------+
|           metformin|TREATMENT|     Present|    0.5364|
|           glipizide|TREATMENT|     Present|    0.9993|
|       dapagliflozin|TREATMENT|     Present|       1.0|
|                 HTG|  PROBLEM|Hypothetical|       1.0|
|Physical examination|     TEST|    Possible|     0.943|
|          tenderness|  PROBLEM|      Absent|       1.0|
|            guarding|  PROBLEM|      Absent|       1.0|
|            rigidity|  PROBLEM|Hypothetical|    0.9999|
+--------------------+---------+------------+----------+


# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like present or conditional; and do not want absent conditions get out of your pipeline.

In [45]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setEnableInMemoryStorage(True)

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_filterer = AssertionFilterer()\
    .setInputCols("sentence","ner_chunk","assertion")\
    .setOutputCol("assertion_filtered")\
    .setCaseSensitive(False)\
    .setWhiteList(["PREsent"])

nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      clinical_assertion,
      assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [46]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])


In [47]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

[('a headache', 'Absent')]


In [48]:
assertion_filterer.getWhiteList()

['PREsent']


In [49]:
chunks=[]
entities=[]
status=[]
confidence=[]

light_result = light_model.fullAnnotate(text)[0]

for m in light_result['assertion_filtered']:

    chunks.append(m.result)
    entities.append(m.metadata['entity'])
    status.append(m.metadata['assertion'])
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

Empty DataFrame
Columns: [chunks, entities, assertion, confidence]
Index: []


As you see, there is no "pain" chunk since it has "absent" assertion label.

# AssertionChunkConverter

In some cases, there may be issues while creating the chunk column by using token indices and losing some data while training and testing the assertion status model if there are issues in these token indices. So we developed a new `AssertionChunkConverter` annotator that takes **begin and end indices of the chunks** as input and creates an extended chunk column with metadata that can be used for assertion status detection model training.

*NOTE*: Chunk begin and end indices in the assertion status model training dataframe can be populated using the new version of ALAB module.

In [51]:
data = spark.createDataFrame([["An angiography showed bleeding in two vessels off of the Minnie supplying the sigmoid that were succesfully embolized.", "Minnie", 57, 63],
     ["After discussing this with his PCP, Leon was clear that the patient had had recurrent DVTs and ultimately a PE and his PCP felt strongly that he required long-term anticoagulation ", "PCP", 31, 34]])\
     .toDF("text", "target", "char_begin", "char_end")

data.show()

+--------------------+------+----------+--------+
|                text|target|char_begin|char_end|
+--------------------+------+----------+--------+
|An angiography sh...|Minnie|        57|      63|
|After discussing ...|   PCP|        31|      34|
+--------------------+------+----------+--------+


In [52]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("tokens")

converter = AssertionChunkConverter() \
    .setInputCols("tokens")\
    .setChunkTextCol("target")\
    .setChunkBeginCol("char_begin")\
    .setChunkEndCol("char_end")\
    .setOutputTokenBeginCol("token_begin")\
    .setOutputTokenEndCol("token_end")\
    .setOutputCol("chunk")

pipeline = Pipeline().setStages([document_assembler,sentenceDetector, tokenizer, converter])

results = pipeline.fit(data).transform(data)

In [53]:
results\
    .selectExpr(
        "target",
        "char_begin",
        "char_end",
        "token_begin",
        "token_end",
        "tokens[token_begin].result",
        "tokens[token_end].result",
        "target",
        "chunk")\
    .show(truncate=False)

+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|target|char_begin|char_end|token_begin|token_end|tokens[token_begin].result|tokens[token_end].result|target|chunk                                         |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|Minnie|57        |63      |10         |10       |Minnie                    |Minnie                  |Minnie|[{chunk, 57, 62, Minnie, {sentence -> 0}, []}]|
|PCP   |31        |34      |5          |5        |PCP                       |PCP                     |PCP   |[{chunk, 31, 33, PCP, {sentence -> 0}, []}]   |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+


# Train a custom Assertion Model

In [52]:
#data source:
#https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv

In [54]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("s3://<bucket>/rawdata/i2b2_assertion_sample_short.csv")

In [55]:
assertion_df.show(3, truncate=100)

+-------------------------------------------------+-------------------+-------+-----+---+
|                                             text|             target|  label|start|end|
+-------------------------------------------------+-------------------+-------+-----+---+
|She has no history of liver disease , hepatitis .|      liver disease| absent|    5|  6|
|                         1. Undesired fertility .|undesired fertility|present|    1|  2|
|                            3) STATUS POST FALL .|               fall|present|    3|  3|
+-------------------------------------------------+-------------------+-------+-----+---+
only showing top 3 rows


In [82]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 721
Test Dataset Count: 170


In [83]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

+-------+-----+
|label  |count|
+-------+-----+
|present|546  |
|absent |175  |
+-------+-----+


In [84]:
test_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

+-------+-----+
|label  |count|
+-------+-----+
|present|117  |
|absent |53   |
+-------+-----+


In [58]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")\
    .setEnableInMemoryStorage(True)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


We will transform our test data with a pipeline consisting of same steps with the pipeline which contains AssertionDLApproach.
By doing this, we enable that test data will have same columns with training data in AssertionDLApproach. <br/>
The goal of this implementation is enabling the usage of `setTestDataset()` parameter in AssertionDLApproach.

In [85]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings])

assertion_test_data = clinical_assertion_pipeline.fit(test_data).transform(test_data)

In [86]:
assertion_test_data.columns

['text', 'target', 'label', 'start', 'end', 'document', 'chunk', 'token', 'embeddings']


We save the test data in parquet format to use in `AssertionDLApproach()`.

In [87]:
# assertion_test_data.write.mode("overwrite").parquet('/tmp/i2b2_assertion_sample_test_data.parquet')

In [97]:
assertion_test_data.write.mode("overwrite").parquet('s3://<bucket>/rawdata/i2b2_assertion_sample_test_data.parquet')

In [98]:
spark.read.parquet('s3://<bucket>/rawdata/i2b2_assertion_sample_test_data.parquet').show()

+--------------------+--------------------+-------+-----+---+--------------------+--------------------+--------------------+--------------------+
|                text|              target|  label|start|end|            document|               chunk|               token|          embeddings|
+--------------------+--------------------+-------+-----+---+--------------------+--------------------+--------------------+--------------------+
|# Hypotension : P...|         hypotensive|present|    5|  5|[{document, 0, 11...|[{chunk, 26, 36, ...|[{token, 0, 0, #,...|[{word_embeddings...|
|(1) Insulin depen...|        foot lesions| absent|   12| 13|[{document, 0, 87...|[{chunk, 74, 85, ...|[{token, 0, 0, (,...|[{word_embeddings...|
|- DM with neuropa...|                  dm|present|    1|  1|[{document, 0, 21...|[{chunk, 2, 3, dm...|[{token, 0, 0, -,...|[{word_embeddings...|
|1. CORONARY ARTER...|  two vessel disease|present|   18| 20|[{document, 0, 12...|[{chunk, 106, 123...|[{token, 0, 0, 1,...|

##  Graph setup

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline.

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [62]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder= "tfgraphs"

assertion_graph_builder = TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(250)\
    .setHiddenUnitsNumber(25)

In [ ]:
'''
# ready to use tf_graph

!mkdir training_logs
!mkdir assertion_tf_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph
'''

In [18]:
'''
# create custom graph

from sparknlp_jsl.training import tf_graph
tf_graph.print_model_params("assertion_dl")

feat_size = 200
n_classes = 6

tf_graph.build("assertion_dl",
              build_params={"n_classes": n_classes},
              model_location= "./tf_graphs",
              model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))
'''

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [99]:
scope_window = [10,10]
ner_logs = "training_logs"

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(64)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(20)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setIncludeConfidence(True)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="s3://<bucket>/rawdata/i2b2_assertion_sample_test_data.parquet/")\
    .setScopeWindow(scope_window)
    

# '''
# If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
# by a pipeline consisting of document, chunk, embeddings stages.
# '''

In [ ]:
'''
assertionStatus = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

In [100]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertion_graph_builder,
    assertionStatus])

In [101]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: tfgraphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 2, 'feat_size': 200, 'max_seq_len': 250, 'n_hidden': 25}
assertion_dl graph exported to tfgraphs/assertion_graph.pb


## Checking the results

Checking the results saved in the log file

In [102]:
import os

log_files = os.listdir("./training_logs")
log_files

['AssertionDLApproach_1e1b18601e30.log', 'AssertionDLApproach_6efc92b020ae.log']


In [103]:
with open("./training_logs/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: tfgraphs/assertion_graph.pb
Training started, trainExamples: 721


Epoch: 0 started, learning rate: 0.001, dataset size: 577
Done, 6.136971336 total training loss: 10.560379, avg training loss: 1.0560379, batches: 10
Quality on validation dataset (20.0%), validation examples = 144
time to finish evaluation: 0.89s
Total validation loss: 1.8190	Avg validation loss: 0.6063
label	 tp	 fp	 fn	 prec	 rec	 f1
present	 90	 25	 21	 0.7826087	 0.8108108	 0.79646015
absent	 8	 21	 25	 0.27586207	 0.24242425	 0.25806448
tp: 98 fp: 46 fn: 46 labels: 2
Macro-average	 prec: 0.52923536, rec: 0.5266175, f1: 0.52792317
Micro-average	 prec: 0.6805556, rec: 0.6805556, f1: 0.6805556


Epoch: 1 started, learning rate: 9.5E-4, dataset size: 577
Done, 3.327187987 total training loss: 7.533205, avg training loss: 0.7533205, batches: 10
Quality on validation dataset (20.0%), validation examples = 144
time to finish evaluation: 0.73s
Total validation loss: 1.7385	Avg validation loss: 

In [104]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+-------+---------+
|  label|   result|
+-------+---------+
|present|[present]|
| absent|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
+-------+---------+
only showing top 20 rows


In [105]:
preds_df = preds.toPandas()

In [106]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
preds_df

       label   result
0    present  present
1     absent  present
2    present  present
3    present  present
4    present  present
..       ...      ...
165  present  present
166   absent   absent
167   absent   absent
168   absent   absent
169  present  present

[170 rows x 2 columns]


In [107]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['label'], preds_df['result']))

              precision    recall  f1-score   support

      absent       0.79      0.79      0.79        53
     present       0.91      0.91      0.91       117

    accuracy                           0.87       170
   macro avg       0.85      0.85      0.85       170
weighted avg       0.87      0.87      0.87       170


In [109]:
spark._jsc.hadoopConfiguration().set("mapred.output.committer.class", "org.apache.hadoop.mapred.DirectFileOutputCommitter")

In [110]:
# save model
assertion_model.stages[-1].write().overwrite().save('s3://<bucket>/models/assertion_custom_model')

## Load saved model

In [114]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [115]:
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.load("s3://<bucket>/models/assertion_custom_model/") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


ner_clinical download started this may take some time.
[OK!]


In [116]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia and pain noted'


light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

print( text  )
print("."*len(text),"\n")

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia and pain noted
........................................................................................................................................ 

       chunks entities assertion confidence
0  a headache  PROBLEM   present     0.9828
1   a head CT     TEST   present     0.9901
2     anxious  PROBLEM   present     0.8746
3    alopecia  PROBLEM    absent     0.8135
4        pain  PROBLEM    absent     0.8224
